In [2]:
import torch
import transformers
!pip install onnx
!pip install onnxruntime
import onnx
import onnxruntime

# Загрузка предобученной модели из hugging face
model_name = "bert-base-uncased"
tokenizer = transformers.BertTokenizer.from_pretrained(model_name)
model = transformers.BertModel.from_pretrained(model_name)

# Создание входных данных для модели
input_text = "Example input text"
inputs = tokenizer(input_text, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Запуск модели и вывод результата
output = model(input_ids, attention_mask=attention_mask)
print("Результат до конвертации в ONNX:", output)

# Конвертация модели в ONNX формат
input_names = ["input_ids", "attention_mask"]
output_names = ["output"]
torch.onnx.export(model, (input_ids, attention_mask), "bert_model.onnx", input_names=input_names, output_names=output_names)

# Запуск конвертированной модели с помощью ONNX Runtime
onnx_session = onnxruntime.InferenceSession("bert_model.onnx")
onnx_inputs = {
    "input_ids": inputs["input_ids"].numpy(),
    "attention_mask": inputs["attention_mask"].numpy()
}
onnx_outs = onnx_session.run(None, onnx_inputs)
print("Результат после конвертации в ONNX и запуска с ONNX Runtime:", onnx_outs)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Результат до конвертации в ONNX: BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2796,  0.0169, -0.2637,  ..., -0.7017,  0.0977,  0.8031],
         [-0.1798,  0.1107, -0.7470,  ..., -0.3447,  0.5448,  0.1668],
         [-1.2174,  0.5254, -0.5268,  ..., -1.1756,  0.0154,  0.6167],
         [-0.1411, -0.0105,  0.0791,  ..., -1.0269, -0.7315,  0.5230],
         [ 0.9685,  0.0289, -0.5429,  ...,  0.2606, -0.7593, -0.2211]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9328, -0.4458, -0.3608,  0.8383,  0.5029, -0.3723,  0.9373,  0.1758,
         -0.2983, -1.0000, -0.2244,  0.8464,  0.9874, -0.0981,  0.9410, -0.7174,
         -0.3811, -0.6312,  0.4028, -0.7821,  0.6708,  0.9997,  0.4789,  0.2977,
          0.5074,  0.9366, -0.7493,  0.9554,  0.9603,  0.7777, -0.8145,  0.3742,
         -0.9941, -0.2757, -0.6069, -0.9931,  0.4635, -0.8161, -0.0954, -0.1169,
         -0.9384,  0.4494,  0.9999, -0.2208,  0.3670, -0.4254, -1.0000,  0.3766,
   

In [3]:
!pip install onnx2torch
from onnx2torch.converter import convert
torch_model = convert("bert_model.onnx")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 689.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [6]:
#output = torch_model(input_ids) - так не работало, исправил
output = torch_model(input_ids, attention_mask)
print(output)

[tensor([[[-0.2796,  0.0169, -0.2637,  ..., -0.7017,  0.0977,  0.8031],
         [-0.1798,  0.1107, -0.7470,  ..., -0.3447,  0.5448,  0.1668],
         [-1.2173,  0.5254, -0.5268,  ..., -1.1756,  0.0154,  0.6167],
         [-0.1411, -0.0105,  0.0791,  ..., -1.0269, -0.7315,  0.5230],
         [ 0.9685,  0.0289, -0.5429,  ...,  0.2606, -0.7593, -0.2211]]],
       grad_fn=<NativeLayerNormBackward0>), tensor([[-0.9328, -0.4458, -0.3608,  0.8383,  0.5029, -0.3723,  0.9373,  0.1758,
         -0.2983, -1.0000, -0.2244,  0.8464,  0.9874, -0.0981,  0.9410, -0.7174,
         -0.3811, -0.6312,  0.4028, -0.7821,  0.6708,  0.9997,  0.4789,  0.2977,
          0.5074,  0.9366, -0.7493,  0.9554,  0.9603,  0.7777, -0.8145,  0.3742,
         -0.9941, -0.2757, -0.6069, -0.9931,  0.4635, -0.8161, -0.0954, -0.1169,
         -0.9384,  0.4494,  0.9999, -0.2208,  0.3670, -0.4254, -1.0000,  0.3766,
         -0.9313,  0.5583,  0.4838,  0.4345,  0.2607,  0.5738,  0.5584, -0.0695,
          0.0196,  0.3151, -0.2